In [1]:
GITHUB_USERNAME = "$GITHUB_USERNAME$"
GITHUB_REF = "$GITHUB_REF$"
NOTEBOOK_TYPE = "$NOTEBOOK_TYPE$"
PYTHON_VERSION = "$PYTHON_VERSION$"
IPYTHON_VERSION = "$IPYTHON_VERSION$"

In [2]:
from pathlib import Path

import requests


if NOTEBOOK_TYPE == 'colab':
    # utils module doesn't exist on colab VM, so get current version from GitHub
    utils_module = Path('utils.py').resolve()
    response = requests.get(f'https://raw.githubusercontent.com/{GITHUB_USERNAME}/davos/{GITHUB_REF}/tests/utils.py')
    utils_module.write_text(response.text)
    # also need to install davos locally
    from utils import install_davos
    install_davos(source='github', ref=GITHUB_REF, fork=GITHUB_USERNAME)

/Users/paxtonfitzpatrick/opt/anaconda3/envs/colab/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
import davos
from IPython.core.inputtransformer import StatelessInputTransformer

from utils import DavosAssertionError, raises, run_tests

In [4]:
IPYTHON_SHELL = get_ipython()

# tests for `davos.implementations.ipython_pre7`
**Note**: this test notebook is only run for jobs where `IPython<7.0`

In [5]:
def test_ipython_pre7_imports():
    """
    check that functions that should've been imported from the 
    ipython_pre7 module came from the right place
    """
    ipy_pre7_funcs = (
        '_activate_helper', 
        '_deactivate_helper',
        'generate_parser_func'
    )
    for func_name in ipy_pre7_funcs:
        func_obj = getattr(davos.implementations, func_name)
        func_module = getattr(func_obj, '__module__')
        assert func_module == 'davos.implementations.ipython_pre7', (
            f"davos.implementations.{func_name} is {func_module}.{func_name}. "
            f"Expected davos.implementations.ipython_pre7.{func_name}"
        )

In [6]:
def test_activate_helper_registers_parser():
    """
    check that the `davos` parser was wrapped in a 
    `StatelessInputTransformer` instance and added to the
    `python_line_transforms` lists of both the `input_splitter` and 
    `input_transformer_manager` instances when `davos` was imported 
    above
    """
    davos_parser = davos.implementations.full_parser
    splitter_xforms = IPYTHON_SHELL.input_splitter.python_line_transforms
    manager_xforms = IPYTHON_SHELL.input_transformer_manager.python_line_transforms
    
    for xform in splitter_xforms:
        if xform.func is davos_parser:
            assert isinstance(xform, StatelessInputTransformer), type(xform)
            break
    else:
        raise DavosAssertionError(
            f"{davos_parser} (as StatelessInputTransformer) not in "
            f"input_splitter list: {splitter_xforms}"
        )
    
    for xform in manager_xforms:
        if xform.func is davos_parser:
            assert isinstance(xform, StatelessInputTransformer), type(xform)
            break
    else:
        raise DavosAssertionError(
            f"{davos_parser} (as StatelessInputTransformer) not in "
            f"input_transformer_manager list: {manager_xforms}"
        )

In [7]:
def test_activate_helper_registers_parser_once():
    """
    `_activate_helper` should not register multiple instances of the 
    `davos` parser if called multiple times (without deactivating)
    """
    smuggle_func = davos.core.core.smuggle
    davos_parser = davos.implementations.full_parser
    splitter_xforms = IPYTHON_SHELL.input_splitter.python_line_transforms
    manager_xforms = IPYTHON_SHELL.input_transformer_manager.python_line_transforms
    
    def count_davos_parsers(xforms_list):
        n = 0
        for xform in xforms_list:
            if xform.func == davos_parser:
                n += 1
        return n
    
    assert count_davos_parsers(splitter_xforms) == 1
    assert count_davos_parsers(manager_xforms) == 1
    
    for _ in range(5):
        davos.implementations.ipython_pre7._activate_helper(smuggle_func, 
                                                            davos_parser)
    
    assert count_davos_parsers(splitter_xforms) == 1
    assert count_davos_parsers(manager_xforms) == 1

In [8]:
def test_activate_helper_adds_smuggle():
    """
    `_activate_helper` should inject the `smuggle_func` it is passed 
    into the IPython user namespace as `"smuggle"`
    """    
    real_smuggle_func = davos.core.core.smuggle
    davos_parser = davos.implementations.full_parser
    
    def _fake_smuggle_func():
        pass
    
    assert smuggle is IPYTHON_SHELL.user_ns['smuggle'] is real_smuggle_func
    
    try:
        davos.implementations.ipython_pre7._activate_helper(_fake_smuggle_func, 
                                                            davos_parser)
        assert smuggle is IPYTHON_SHELL.user_ns['smuggle'] is _fake_smuggle_func
    finally:
        # regardless of test outcome, make sure original value of 
        # "smuggle" is restored
        IPYTHON_SHELL.user_ns['smuggle'] = real_smuggle_func

In [9]:
def test_deactivate_helper_removes_parser():
    """
    `_deactivate_helper` should remove the `StatelessInputTransformer` 
    whose `.func` is the `davos` parser from both `input_splitter` and 
    `input_transformer_manager`
    """
    smuggle_func = davos.core.core.smuggle
    davos_parser = davos.implementations.full_parser
    # references
    splitter_xforms = IPYTHON_SHELL.input_splitter.python_line_transforms
    manager_xforms = IPYTHON_SHELL.input_transformer_manager.python_line_transforms
    # copies
    old_splitters_list = IPYTHON_SHELL.input_splitter.python_line_transforms[:]
    old_managers_list = IPYTHON_SHELL.input_transformer_manager.python_line_transforms[:]
    
    def count_davos_parsers(xforms_list):
        n = 0
        for xform in xforms_list:
            if xform.func == davos_parser:
                n += 1
        return n
    
    assert count_davos_parsers(splitter_xforms) == 1
    assert count_davos_parsers(manager_xforms) == 1
        
    try:
        davos.implementations.ipython_pre7._deactivate_helper(smuggle_func, 
                                                              davos_parser)

        assert count_davos_parsers(splitter_xforms) == 0, splitter_xforms
        assert count_davos_parsers(manager_xforms) == 0, manager_xforms
    finally:
        # regardless of test outcome, make sure both original 
        # transformer lists and value of "smuggle" are restored
        IPYTHON_SHELL.input_splitter.python_line_transforms = old_splitters_list
        IPYTHON_SHELL.input_transformer_manager.python_line_transforms = old_managers_list
        IPYTHON_SHELL.user_ns['smuggle'] = smuggle_func

In [10]:
def test_deactivate_helper_deletes_smuggle():
    """
    `_deactivate_helper` should remove `"smuggle"` from the IPython user 
    namespace if it refers to `davos.core.core.smuggle`
    """
    smuggle_func = davos.core.core.smuggle
    davos_parser = davos.implementations.full_parser
    old_splitters_list = IPYTHON_SHELL.input_splitter.python_line_transforms[:]
    old_managers_list = IPYTHON_SHELL.input_transformer_manager.python_line_transforms[:]
    
    assert smuggle is IPYTHON_SHELL.user_ns['smuggle'] is smuggle_func
    
    try:
        davos.implementations.ipython_pre7._deactivate_helper(smuggle_func, 
                                                              davos_parser)
        assert 'smuggle' not in IPYTHON_SHELL.user_ns
        with raises(NameError):
            smuggle os

    finally:
        IPYTHON_SHELL.user_ns['smuggle'] = smuggle_func
        IPYTHON_SHELL.input_splitter.python_line_transforms = old_splitters_list
        IPYTHON_SHELL.input_transformer_manager.python_line_transforms = old_managers_list

In [11]:
def test_deactivate_helper_leaves_smuggle():
    """
    running `_deactivate_helper` should *not* delete `"smuggle"` from 
    the IPython user namespace if it refers to something other than 
    `davos.core.core.smuggle`
    """
    global smuggle
    
    smuggle_func = davos.core.core.smuggle
    davos_parser = davos.implementations.full_parser
    old_splitters_list = IPYTHON_SHELL.input_splitter.python_line_transforms[:]
    old_managers_list = IPYTHON_SHELL.input_transformer_manager.python_line_transforms[:]
    
    smuggle = 'tmp-smuggle-val'
    try:
        davos.implementations.ipython_pre7._deactivate_helper(smuggle_func, 
                                                              davos_parser)
        assert 'smuggle' in IPYTHON_SHELL.user_ns
        assert smuggle == 'tmp-smuggle-val'
    finally:
        IPYTHON_SHELL.user_ns['smuggle'] = smuggle_func
        IPYTHON_SHELL.input_splitter.python_line_transforms = old_splitters_list
        IPYTHON_SHELL.input_transformer_manager.python_line_transforms = old_managers_list

In [12]:
def test_deactivate_helper_not_active():
    """
    running `_deactivate_helper` shouldn't cause any problems if run 
    when the davos parser is already inactive
    """
    smuggle_func = davos.core.core.smuggle
    davos_parser = davos.implementations.full_parser
    
    davos.deactivate()

    try:
        for _ in range(5):
            davos.implementations.ipython_pre7._deactivate_helper(smuggle_func, 
                                                                  davos_parser)
    finally:
        davos.activate()

In [13]:
def test_generate_parser_func_noop():
    """
    the `IPython<7.0` implementation fo `generate_parser_func` should 
    simply return the single-line parser, untouched
    """
    davos_parser = davos.implementations.full_parser
    line_parser = davos.core.core.parse_line
    assert davos_parser is line_parser

In [14]:
run_tests()

collected 9 items



test_ipython_pre7_imports PASSED

test_activate_helper_registers_parser PASSED

test_activate_helper_registers_parser_once PASSED

test_activate_helper_adds_smuggle PASSED

test_deactivate_helper_removes_parser PASSED

test_deactivate_helper_deletes_smuggle PASSED

test_deactivate_helper_leaves_smuggle PASSED

test_deactivate_helper_not_active PASSED

test_generate_parser_func_noop PASSED